In [5]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

In [6]:
symbol = "INFY"

def get_historical_stock_price(symbol, to_date = datetime.today().strftime("%d-%m-%Y"), 
                                   from_date = (datetime.today() - timedelta(days = 367)).strftime("%d-%m-%Y")):
    df = equity_history(symbol, "EQ", from_date, to_date)
    df = df.drop(columns = ['_id','CH_SYMBOL','CH_SERIES','CH_MARKET_TYPE', 'CH_LAST_TRADED_PRICE',
                                    'CH_PREVIOUS_CLS_PRICE','CH_TOTAL_TRADES','TIMESTAMP','createdAt',
                                    'updatedAt','__v','VWAP','mTIMESTAMP','CA'])
    df.columns = ['High', 'Low', 'Open', 'Close', 'Total Traded Quantity', 'Total Traded Value', 
                  '52 week High', '52 week Close', 'ISIN', 'Date']
    return df.set_index('Date')

In [7]:
def engulfing(data, x_days = 1):
    results = []
    data['prev_candle_body_ratio'] = data['candle_body_ratio'].shift(1)
    data['prev_candle_color'] = data['candle_color'].shift(1)
    data['prev_candle_body_length'] = data['candle_body_length'].shift(1)
    data['prev_Close'] = data['Close'].shift(1)
    data['prev_Open'] = data['Open'].shift(1)
    data['prev_current_body_ratio'] = data['prev_candle_body_length'] / data['candle_body_length']
    return np.where(np.logical_and.reduce([data['prev_candle_body_ratio'] >= 15, data['prev_candle_color'] == "Red", data['candle_color'] == "Green", data['Close'] > data['prev_Open'], data['Open'] < data['prev_Close'], data['prev_current_body_ratio'] < .9]), "Bullish Engulfing", 
                   np.where(np.logical_and.reduce([data['prev_candle_body_ratio'] >= 15, data['prev_candle_color'] == "Green", data['candle_color'] == "Red", data['Open'] > data['prev_Close'], data['Close'] < data['prev_Open'], data['prev_current_body_ratio'] < .9]), "Bearish Engulfing", ""))

In [9]:
data = get_historical_stock_price(symbol)

In [10]:
data['candle_body_length'] = (data['Close'] - data['Open']).abs()
data['candle_body_ratio'] = (data['candle_body_length'] * 100 / (data['High'] - data['Low']))
data['candle_length'] = data['High'] - data['Low']
data['candle_color'] = np.where(data['Open'] > data['Close'], "Red", "Green") 

In [11]:
data['engulf?'] = engulfing(data)